## Source: Dota 2 game

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) is a multiplayer computer game of the genre [MOBA](https://ru.wikipedia.org/wiki/MOBA). Players play matches between themselves. In each match two teams participate, 5 people in each. One team plays for the bright side (The Radiant), the other for the dark (The Dire). The goal of each team is to destroy the main building of the enemy base (throne).

There are [different game modes](http://dota2.gamepedia.com/Game_modes/ru), we will consider the mode [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), in the format of which most of the Dota 2 e-sports events take place.

### How is the match going

#### 1. Players choose the heroes

In total, the game has just over 100 different heroes (characters). At the beginning of the game, the teams in a certain order choose the heroes themselves and prohibit the choice of certain heroes to the enemy (bans). Each player will manage one hero, within the same match there can not be several identical heroes. Heroes differ in their characteristics and abilities. The success of the team largely depends on the combination of the chosen characters.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Action

Players can receive gold and experience for killing other characters or other units. Accumulated experience affects the level of the hero, which in turn allows you to improve abilities. For the accumulated gold, players buy items that improve the characteristics of the characters or give them new abilities.

After death, the hero goes to the "tavern" and is reborn only after some time, so the team loses the player for a while, but the player can buy the hero out of the tavern for a certain amount of gold ahead of time.

During the game, the teams develop their heroes, defend their part of the field and attack the enemy.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. The end

The game ends when one of the teams destroys a certain number of enemy "towers" and destroys the throne.

![](http://imgur.com/Du79Kzf.jpg)

## Task is to predict the winner by first 5 minutes of the game

Who will be the winner - Radiant or Dire?

## Dataset

Dataset with matches' data - `matches.jsonlines.bz2`.
If you need to decode some identificators from matches look at `dictionaries` directory.

#### Read the data

Information about matches is recorded in a compressed text file `matches.jsonlines.bz2`, each line of which contains an object in the format [JSON](https://ru.wikipedia.org/wiki/JSON). A JSON record is converted into a python object using the standard `json` module. Example of reading matches:

In [5]:
import json
import bz2

with bz2.BZ2File('./matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)
        
        break

#### Match object description

```python
{
    "match_id": 247,            # id
    "start_time": 1430514316,   # match start time in format of unixtime
    "lobby_type": 0,            # the room's type(can be decoded using dictionaries/lobbies.csv)
 
    # heroes picking
    "picks_bans": [
        {
            "order": 0,       # an action's order
            "is_pick": false, # true if a hero was picked, false — if a hero was banned
            "team": 1,        # team (0 — Radiant, 1 — Dire)
            "hero_id": 95     # hero id(can be decoded using dictionaries/heroes.csv)
        }, 
        ...
    ],

    # every player's info
    # Players with indexes from 0 to 4 — Radiant, from 5 to 9 — Dire
    "players": [ 
        { 
        
            # hero id(can be decoded using dictionaries/heroes.csv)
            "hero_id": 67,  

            # timelines (ticks are set in "times")
            "xp_t": [0, 13, 115, 177, 335, ...],   # experience
            "gold_t": [0, 99, 243, 343, 499, ...], # gold + cost of all the gear bought (net worth)
            "lh_t": [0, 0, 2, 2, 2, ...],          # killed units(not heroes)

            # events list: a heroes' abilities
            "ability_upgrades": [
                {
                    "time": 51,      # game time
                    "level": 1,      # level when the event is done
                    "ability": 5334  # ability(can be decoded using dictionaries/abilities.csv)
                }, 
                ...
            ],

            # events list: kills
            "kills_log": [
                {
                    "time": 831,    # game time
                    "player": 7,    # killed player's index(empty if it was killed not a hero)
                    "unit": "npc_dota_hero_viper" # killed unit type
                }, 
                ...
            ],

            # events list: buying items
            "purchase_log": [
                {
                    "time": -73,     # game time(can be negative as the 0 is when the units are spawned). Before 0 you have some time to buy gear.
                    "item_id": 44    # item id(can be decoded using dictionaries/items.csv)
                }, 
                ...
            ]

            # events list: buybacks
            "buyback_log": [
                {"time": 2507},
                ...
            ],

            # events list: observers which can monitor a part of the game's map near the set point
            "obs_log": [
                {
                    "time": 1711,    # game time
                    "xy": [111, 130] # set point's coordinates
                }, 
                ...
            ],
            "sen_log": [], # like obs_log, but another observer's type

        },
        ...
    ],
    
    # ticks for timelines
    "times": [0, 60, 120, 180, ...],

    # key events list
    "objectives": [
        {
            "time": 198,           # game time
            "type": "firstblood",  # type
            "player1": 6,          # event's features
            "player2": 1           #   players' indexes, 
                                   #   team (0 — Radiant, 1 — Dire)
        }, 
        {
            "time": 765, 
            "type": "tower_kill", 
            "player": 7, 
            "team": 1
        }, 
        ...
    ]
    
    # the game's total
    "finish": {
        "duration": 2980,             # duration in seconds
        "radiant_win": false,         # true, if Radiant won
        "tower_status_radiant": 0,    # towers' condition
        "tower_status_dire": 1972,    
        "barracks_status_dire": 63,   # barracks' condition
        "barracks_status_radiant": 0
    }
}
```

#### Towers' and barracks' condition

The state of the towers by the end of the game is given by an integer, encoded in bits:

```
┌─┬─┬─┬─┬─────────────────────── Not used.
│ │ │ │ │ ┌───────────────────── Ancient Bottom
│ │ │ │ │ │ ┌─────────────────── Ancient Top
│ │ │ │ │ │ │ ┌───────────────── Bottom Tier 3
│ │ │ │ │ │ │ │ ┌─────────────── Bottom Tier 2
│ │ │ │ │ │ │ │ │ ┌───────────── Bottom Tier 1
│ │ │ │ │ │ │ │ │ │ ┌─────────── Middle Tier 3
│ │ │ │ │ │ │ │ │ │ │ ┌───────── Middle Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ ┌─────── Middle Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ ┌───── Top Tier 3
│ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─── Top Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─ Top Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
```

The state of the barracks at the end of the game is encoded in bits of the integer:
```
┌─┬───────────── Not used.
│ │ ┌─────────── Bottom Ranged
│ │ │ ┌───────── Bottom Melee
│ │ │ │ ┌─────── Middle Ranged
│ │ │ │ │ ┌───── Middle Melee
│ │ │ │ │ │ ┌─── Top Ranged
│ │ │ │ │ │ │ ┌─ Top Melee
│ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0
```

## Extracting features

The `extract_features.py` script extracts features from the known information about the match in the first 5 minutes of the game, makes a table of them. The table will help you to quickly form the matrix object-feature, the vector of answers and begin to apply machine learning methods to solve the problem.

The attributes presented in the `features.csv` table are, in the opinion of domain experts, the most important for solving the task of predicting team victory. However, it is not necessary to use these signs in their original form to apply machine learning methods — you can make new signs from existing ones. Moreover, the signs in the `features.csv` file do not contain all the information known about the match in the first 5 minutes of the game. You can use the `extract_features.py` script as an example and add your features to improve the quality of the prediction.

#### Features example

In [3]:
import pandas
features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


#### Features' description

- `match_id`: match id
- `start_time`: start time (unixtime)
- `lobby_type`: the room's type(can be decoded using dictionaries/lobbies.csv)
- Feature for every player (Radiant prefixes - `rN`, Dire prefixes — `dN`):
    - `r1_hero`: hero (can be decoded using dictionaries/heroes.csv)
    - `r1_level`: max herol level for the first 5 minutes of a game
    - `r1_xp`: max experience earned
    - `r1_gold`: max gold earned
    - `r1_lh`: killed units count
    - `r1_kills`: killed heroes count
    - `r1_deaths`: deaths count
    - `r1_items`: bought items count
- "First blood" features. If there was no "first blood" the values would be None.
    - `first_blood_time`: time of the "first blood"
    - `first_blood_team`: team got "first blood" (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: hero assisted
    - `first_blood_player2`: hero 2 assisted
- Team's features (prefixes are `radiant_` and `dire_`)
    - `radiant_bottle_time`: time of the first "bottle" item bought
    - `radiant_courier_time`: time of the "courier" item bought
    - `radiant_flying_courier_time`: time of the "flying courier" item bought
    - `radiant_tpscroll_count`: "tpscroll" items bought in first 5 minutes
    - `radiant_boots_count`: "boots" items bought
    - `radiant_ward_observer_count`: "ward observer" items bought
    - `radiant_ward_sentry_count`: "ward sentry" items bought
    - `radiant_first_ward_time`: first ward time
- Match's total(there is no total in our data as it's out of the first 5 minutes of a game)
    - `duration`: duration
    - `radiant_win`: 1 if Radiant won, 0 otherwise
    - Towers' and barracks' condition
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

## Quality metric
As a quality metric, we will use the area under the ROC curve (AUC-ROC). Note that the AUC-ROC is a quality metric for an algorithm that gives out first class grades. Both algorithms that will be used in the project - gradient boosting and logistic regression - are able to produce such estimates. For this you need to get predictions using the function predict_proba. It returns two columns: the first contains estimates of belonging to the zero class, the second - the first class. You need values from the second column:
```python
pred = clf.predict_proba(X_test)[:, 1]
```

## Analysis

### Way 1: Gradient boosting
One of the most versatile algorithms is gradient boosting. It is not very demanding on data, restores non-linear dependencies, and works well on many data sets, which determines its popularity. It is a reasonable idea to try it for him in the first place. Checkpoints to be reached:

1. Read the table with the tags from the features.csv file using the code above. Remove the attributes associated with the outcome of the match (they are marked in the data description as missing in the test set).
2. Check the sample for gaps using the count () function, which for each column shows the number of filled values. Are there many gaps in the data? Write down the names of the signs that have omissions, and try for any two of them to give a reason why their values may be missing.
3. Replace blanks with zeros using the fillna () function. In fact, this method is preferable for logistic regression, since it will allow the missing value to make no contribution to the prediction. For trees, often the best option is to replace the pass with a very large or very small value - in this case, when building a vertex partition, you can send objects with gaps to a separate branch of the tree. There are also other approaches - for example, replacing a pass with an average value of a sign. We do not require this in the task, but if you wish, try different approaches to the processing of passes and compare them with each other.
3. Which column contains the target variable? Write down its name.
4. Forget that there are categorical signs in the sample, and we will try to train the gradient boosting over the trees on the existing "objects-features" matrix. Fix the split generator for cross-validation of 5 blocks (KFold), do not forget to mix the sample (shuffle = True), because the data in the table is sorted by time, and without mixing you can face undesirable effects when evaluating the quality. Assess the quality of the gradient boosting (GradientBoostingClassifier) using this cross-validation, try a different number of trees (at least test the following values for the number of trees: 10, 20, 30). How long have classifiers been configured? Is the optimum at the tested values of the n_estimators parameter, or is the quality likely to continue to increase with its further increase?

##### Questions to be answered
1. What signs have omissions among their values? What can the gaps in these signs mean (answer this question for any two signs)?
2. What is the name of the column containing the target variable?
3. How long was cross-validation done for gradient boosting with 30 trees? Instructions for measuring time can be found below. What is the quality with this? Recall that in this assignment we use the AUC-ROC quality metric.
4. Does it make sense to use more than 30 trees in a gradient boost? What would you suggest to do to speed up his training while increasing the number of trees?

### Way 2: Logistic Regression

Linear methods work much faster than tree compositions, so it seems reasonable to use them to speed up data analysis. One of the most common methods for classification is logistic regression. Checkpoints to be reached:

1. Assess the quality of the logistic regression (sklearn.linear_model.LogisticRegression with L2 regularization) using cross-validation using the same scheme that was used for gradient boosting. Pick the best regularization parameter (C). What is the best quality you got? How does it compare to the quality of the gradient boost? How can you explain this difference? Is logistic regression faster than gradient boosting?
2. Among the signs in the sample there are categorical ones that we used as numerical ones, which is hardly a good idea. The categorical features in this problem are eleven: lobby_type and r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Remove them from the sample, and cross-validate for logistic regression on the new sample with the selection of the best regularization parameter. Has the quality changed? How can you explain this?
3. In the previous step, we removed the rM_hero and dM_hero signs from the sample, which show which heroes played for each team. These are important signs - heroes have different characteristics, and some of them win more often than others. Find out from the data how many different hero identifiers exist in this game (you may need the unique or value_counts function).
4. We use the "bag of words" approach to encode information about the characters. Let there be N different heroes in the game. We form N signs, with the i-th being equal to zero if the i-th hero did not participate in the match; a unit if the i-th hero played for the Radiant team; minus one if i-th hero played for Dire team. Below you can find the code that performs this conversion. Add the resulting characters to the numeric characters that you used in the second paragraph of this stage.
5. Cross-validate the logistic regression on the new sample with the selection of the best regularization parameter. What is the quality? Has it improved? How can you explain this?
6. Build a prediction of the probability of winning the Radiant team for a test sample using the best model studied (the best in terms of AUC-ROC cross-validation). Make sure that the predicted probabilities are adequate - they are located on the interval [0, 1], do not coincide with each other (ie, the model did not turn out to be constant).

##### Questions to be answered:
1. What is the quality of the logistic regression over all the original features? How does it compare to the quality of the gradient boost? How can you explain this difference? Is logistic regression faster than gradient boosting?
2. How does the removal of categorical features affect the quality of logistic regression (indicate the new value of the quality metric)? How can you explain this change?
3. How many different hero identifiers exist in this game?
4. What is the quality when adding a "bag of words" for the characters? Has it improved from the previous version? How can you explain this?
5. What is the minimum and maximum value of the forecast on the test sample obtained from the best of the algorithms?

## Data reference
[YASP 3.5 Million Data Dump](http://academictorrents.com/details/5c5deeb6cfe1c944044367d2e7465fd8bd2f4acf)